In [1]:
import cv2
import numpy as np
from collections import Counter

In [2]:
#Reading the gray scale image
gray_img = cv2.imread('Q4_gray.jpeg', 2)

In [3]:
gray_img = cv2.resize(gray_img, (900, 600))

In [4]:
#Finding the counts of different intensity values
list_intensity = []
for i in range(gray_img.shape[0]):
    for j in range(gray_img.shape[1]):
        list_intensity.append(gray_img[i][j])

count_pixels = len(list_intensity)

In [5]:
dict_intensity = Counter(list_intensity)
list_pairs = [[a, dict_intensity[a]] for a in dict_intensity]
list_pairs.sort(key = lambda x:x[0])

In [6]:
dict_within_variance = {}
#Finding the within class variance for all possible values of threshold
for k in range(len(list_pairs)):
    #Finding variance of background
    total_wb = 0
    mean_b = 0
    for m in range(k):
        total_wb += list_pairs[m][1]
        mean_b += (list_pairs[m][0] * list_pairs[m][1])
        
    sum_wb = total_wb / count_pixels
    if total_wb == 0:
        mean_b = 0
    else:
        mean_b = mean_b / total_wb
    
    variance_b = 0
    for n in range(k):
        variance_b = variance_b + (pow((list_pairs[n][0] - mean_b), 2) * list_pairs[n][1])
    variance_b = variance_b / count_pixels
    
    #Now finding the values for foreground
    total_wf = 0
    mean_f = 0
    for o in range(k, len(list_pairs)):
        total_wf += list_pairs[o][1]
        mean_f += (list_pairs[o][0] * list_pairs[o][1])
    
    sum_wf = total_wf / count_pixels
    if total_wf == 0:
        mean_f = 0
    else:
        mean_f = mean_f / total_wf
    
    variance_f = 0
    for p in range(k, len(list_pairs)):
        variance_f = variance_f + (pow((list_pairs[p][0] - mean_f), 2) * list_pairs[p][1])
    variance_f = variance_f / count_pixels
    
    #Finding the within class variance and storing in dictionary
    value_within_class = (sum_wb * variance_b) + (sum_wf * variance_f)
    
    dict_within_variance[list_pairs[k][0]] = value_within_class



In [7]:
#Sorting the threshold values according to the within class variance values
list_within_pairs = [[q, dict_within_variance[q]] for q in dict_within_variance]
list_within_pairs.sort(key = lambda x:x[1])
value_threshold = list_within_pairs[0][0]

In [8]:
#Now making the pixels with value < threshold as background and others as foreground
bin_img = np.zeros((600, 900))
for q in range(gray_img.shape[0]):
    for r in range(gray_img.shape[1]):
        if gray_img[q][r] < value_threshold:
            bin_img[q][r] = 0
        else:
            bin_img[q][r] = 255


In [9]:
#Showing the converted binary image
cv2.imshow('binary', bin_img)
cv2.waitKey(0)
cv2.destroyAllWindows()